In [1]:
from kliff.models import KIMModel
from kliff.dataset import Dataset
from kliff.ml.opt import OptimizerScipy
from scipy.optimize import minimize

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# dataset = Dataset(colabfit_database="colabfit_database", colabfit_dataset="jax_si")
dataset = Dataset("/home/openkim/MACH2023/data/Si_training_set_4_configs")

2023-04-06 15:57:30.394 | INFO     | kliff.dataset.dataset:_read:687 - 4 configurations read from /home/openkim/MACH2023/data/Si_training_set_4_configs


### Install the require model/model driver.

In [3]:
!kim-api-collections-management install user "SW_StillingerWeber_1985_Si__MO_405512056662_005"

Item 'SW_StillingerWeber_1985_Si__MO_405512056662_005' already installed in collection 'user'.

Success!


### Use ASE KIMModel class for generating a Python based KIM Model

In [4]:
model = KIMModel(model_name="SW_StillingerWeber_1985_Si__MO_405512056662_005")

### Set parameters for optimization

In [5]:
model.set_opt_params(A=[[10.0]], B=[[0.5]])

In [6]:
model(dataset[0],compute_forces=False)

{'energy': -200.55372425795346, 'forces': None, 'stress': None}

### Explicit Optimization

In [7]:
param_to_optimize = model.parameters()
def loss_fn(params):
    for new_param, old_param in zip(params, param_to_optimize):
        model.copy_parameters(old_param, new_param)
    loss = 0
    for config in dataset:
        E = model(config)["energy"]
        loss += (config.energy - E)**2
    return loss

In [8]:
x0 = [10.0,5.5]
result = minimize(loss_fn, x0, method="L-BFGS-B", tol=1e-12, options={"maxiter":100})

In [9]:
result.x

array([15.28515917,  0.60219947])

### Implicit Optimization

In [10]:
opt = OptimizerScipy(model, param_to_optimize, dataset, optimizer="L-BFGS-B",target_property=["energy"],)
result_im = opt.minimize()

2023-04-06 15:57:43.160 | INFO     | kliff.ml.opt:minimize:157 - Starting with method L-BFGS-B


In [11]:
result_im.x

array([15.28515889,  0.60219945])

In [13]:
model.write_kim_model()

2023-04-06 15:58:14.891 | INFO     | kliff.models.kim:write_kim_model:695 - KLIFF trained model write to `/home/openkim/cwd/SW_StillingerWeber_1985_Si__MO_405512056662_005_kliff_trained`
